# SO the trick I think is= 
initialize the "array" in another process
this allows high flexibility, since that other process can have more complex functions behind the parameters passed in

this would be a beautiful way to set up the size based allometries for example

and that way the current state can be calculated within that process

the only remaining question is then: how to pass state to fluxes.. (perhaps need new argument here? any_object? _model_context?_

In [ ]:
# this is the central state (i think)
@xs.process
class SurfaceTopography:
    """Update the elevation of the (land and/or submarine) surface
    topography.
    """
    elevation = xs.variable(
        dims=('y', 'x'),
        intent='inout',
        description='surface topography elevation'
    )

    motion_upward = xs.foreign(TotalVerticalMotion, 'surface_upward')

    def finalize_step(self):
        self.elevation += self.motion_upward
        


In [ ]:
# this is where it get's initialized in a seperate process via xs.foreign
@xs.process
class FlatSurface:
    """Initialize surface topography as a flat surface at sea-level with
    random perturbations (white noise).
    """
    shape = xs.foreign(UniformRectilinearGrid2D, 'shape')
    elevation = xs.foreign(SurfaceTopography, 'elevation', intent='out')

    def initialize(self):
        self.elevation = np.random.rand(*self.shape)

        # interesting rename:
@xs.process
class BareRockSurface:
    """Initialize topographic surface as a bare rock surface."""

    surf_elevation = xs.foreign(SurfaceTopography, 'elevation')
    bedrock_elevation = xs.foreign(Bedrock, 'elevation', intent='out')

    def initialize(self):
        self.bedrock_elevation = self.surf_elevation.copy()

In [ ]:
# this is kind of interesting, so here a new variable is created
# not sure how it get's reused after here, though
@xs.process
class SurfaceAfterTectonics(SurfaceToErode):
    """Used for the computation erosion processes after
    applying tectonic forcing.
    """
    topo_elevation = xs.foreign(SurfaceTopography, 'elevation')

    forced_motion = xs.foreign(TectonicForcing, 'surface_upward')

    elevation = xs.variable(
        dims=('y', 'x'),
        intent='out',
        description='surface elevation before erosion'
    )

    def run_step(self):
        self.elevation = self.topo_elevation + self.forced_motion


# SO interestingly, in the package fastscape..

it seems like the "Top Level Summation" class TotalVerticalMotion returns 2 variables, and computes these from groups, but otherwise is not referenced
this reminds me of the problem that i had earlier, that crashed my draft
this was that the state of Phytplankton was referenced by the flux, and that the flux needed the state to compute, while the state was in turn computed by the flux

can this be circumvented, by using different steps? like run and finalize?


In [1]:
# This definitely also makes sense, but just as construction help for future components (and building block of the current ones)
@xs.process
class Time:
    days = xs.variable(dims='time', description='time in days')
    # for indexing xarray IO objects
    # NOTE: this is what allows storing the output along time properly even when using odeint!
    time = xs.index(dims='time', description='time in days')
    
    def initialize(self):
        self.time = self.days

@xs.process
class Component:
    """Base class encoding behaviour for a component of a ecosystem.
        subclass needs to have implemented:
        self.num, self.component_label, self.init, 
        
    """
    def initialize(self):
        if self.num:
            self.label = np.array([f"{self.component_label}-{i+1}" for i in range(self.num)])
            self.init = np.array([self.initVal for i in range(self.num)])
        else:
            self.label = self.component_label
            self.init = self.initVal
        self.c_state = self.init
        print(self.label)
        print(self.init)
        
    def run_step(self):
        self._delta_c = sum((v for v in self.c_fluxes))

    def finalize_step(self):
        self.c_state += self._delta_c

@xs.process
class Nutrient(Component):
    """"""
    component_label = xs.variable(default='N', groups='component-label', static=True)
    label = xs.index(dims='N')
    init = xs.variable(dims='N',intent='out')
    
    num = xs.variable(intent='in', static=True, description='number of Phytoplankton')
    initVal = xs.variable(default=1)
    
    fluxes = xs.group('n_flux')
    state = xs.variable(dims='N', groups='state-variable', intent='out', description='component array', attrs={'units': 'µM N'})

            
@xs.process
class Phytoplankton(Component):
    """"""
    component_label = xs.variable(default='P', groups='component-label', static=True)
    label = xs.index(dims='P')
    init = xs.variable(dims='P',intent='out')
    
    initVal = xs.variable(default=1)
    
    fluxes = xs.group('p_flux')
    state = xs.variable(dims='P', groups='state-variable', intent='out', description='component array', attrs={'units': 'µM N'})
    
    # PARAMETERS
    num = xs.variable(intent='in', groups='parameters', static=True, description='number of Phytoplankton')
    halfsat = xs.variable(dims=[(), 'P'], groups='parameters')

@xs.process
class PhytoplanktonGrowth:
    """"""
    growth = xs.variable(
        dims='P',
        intent='out',
        groups=['p_flux'],
        description='phytoplankton growth, under light & nutrient limitation'
    )
    
@xs.process
class NutrientUptake(PhytoplanktonGrowth):
    """"""
    N = xs.foreign(Nutrient, 'state')
    P = xs.foreign(Phytoplankton, 'state')
    P_halfsat = xs.foreign(Phytoplankton, 'halfsat')
    
    def run_step(self):
        self.growth = self.N / (self.P_halfsat + self.N) * self.P
    
    
@xs.process
class Forcing:
    """Sum up all forcing processes and their effects, respectively.
    """
    pass
    

class NutrientSource(Forcing):
    """Chemostat constant nutrient inflow from one direction"""
    forcing_flux = xs.variable(
        dims=[(), 'N'],
        intent='out',
        groups='n_flux',
        description='constant nutrient influx'
    )

    #N = xs.foreign(Nutrient, 'state')
    
    #@xs.runtime(args='step_delta')
    def run_step(self):
        self.forcing_flux = 0.1


NameError: name 'xs' is not defined

In [65]:
xs.Model({'N':Nutrient, 'P':Phytoplankton, 'N_Uptake':NutrientUptake, 'N_source':NutrientSource})

<xsimlab.Model (4 processes, 7 inputs)>
N
    num                 [in] number of Phytoplankton
    component_label     [in]
    initVal             [in]
P
    halfsat             [in] () or ('P',) 
    num                 [in] number of Phytoplankton
    component_label     [in]
    initVal             [in]
N_Uptake
N_source

# SO the trick I think is= 
initialize the "array" in another process
this allows high flexibility, since that other process can have more complex functions behind the parameters passed in

this would be a beautiful way to set up the size based allometries for example

and that way the current state can be calculated within that process

the only remaining question is then: how to pass state to fluxes.. (perhaps need new argument here? any_object? _model_context?_

In [70]:
# this needs to be the NPZD model array

# i think this could flatten each class being initialized.. but does it need to?
@xs.process
class NPZD_1D:
    """Create a 1-dimensional, array of NPZD component states and labels."""

    numberofcomps = xs.variable(description="total length", static=True) # here all the label & state inputs enter, and they get flattened below, and assigned
    
    components = xs.index(dims="NPZD") #x!

    def initialize(self):
        self.components = np.arange(0., self.numberofcomps)

In [71]:
@xs.process
class ProfileC:
    """Compute the evolution of the profile of quantity `u`."""

    c_fluxes = xs.group("c_flux")
    c_state = xs.variable(
        dims="NPZD", intent="inout", description="component array", attrs={"units": "µM N"}
    )

    def run_step(self):
        self._delta_c = sum((v for v in self.c_fluxes))
        
        #print('hey',list(self.u_vars))

    def finalize_step(self):

        self.c_state += self._delta_c

In [72]:
# so the thing is that I want components to be multidimensional:

@xs.process
class NutrientUptake:
    """Advection using finite difference (Lax method) on
    a fixed grid with periodic boundary conditions.

    """

    v = xs.variable(dims=[(), "c"], description="velocity")
    #grid_spacing = xs.foreign(UniformGrid1D, "spacing")
    
    c_state = xs.foreign(ProfileC, "c_state")
    
    c_growth = xs.variable(dims="x", intent="out", groups="c_flux")

    @xs.runtime(args="step_delta")
    def run_step(self, dt):
        #factor = self.v / (2 * self.grid_spacing)

        #q_left = np.roll(self.q, 1)
        #q_right = np.roll(self.q, -1)
        #q_1 = 0.5 * (q_right + q_left) - factor * dt * (q_right - q_left)
        growth= self.v/10
        
        self.c_growth = growth * self.c_state

In [73]:
@xs.process
class InitCStates:
    """Initialize `u` profile using a Gaussian pulse."""

    components = xs.foreign(NPZD_1D, "components")
    c_state = xs.foreign(ProfileC, "c_state", intent="out")

    def initialize(self):
        self.c_state = self.components

In [75]:
advect_model = xs.Model(
    {
        "grid": NPZD_1D,
        "profile": ProfileC,
        "init": InitCStates,
        "advect": NutrientUptake,
    }
)

In [76]:
in_ds = xs.create_setup(
     model=advect_model,
     clocks={
         'time': np.linspace(0., 1., 1001)#,
         #'otime': [0, 0.1, 0.5, 1]
     },
     master_clock='time',
     input_vars={
         'grid__numberofcomps':3,
         'advect__v': 1.
     },
     output_vars={
         'profile__c_state': 'time'
     }
 )

In [77]:
from xsimlab.monitoring import ProgressBar

with ProgressBar(frontend='console'):
    out_ds = in_ds.xsimlab.run(model=advect_model)

██████████ 100% | Simulation finished in 00:01 


In [78]:
out_ds

<xarray.Dataset>
Dimensions:              (NPZD: 3, time: 1001)
Coordinates:
  * time                 (time) float64 0.0 0.001 0.002 ... 0.998 0.999 1.0
Dimensions without coordinates: NPZD
Data variables:
    advect__v            float64 1.0
    components           (NPZD) float64 0.0 2.47e+41 4.94e+41
    grid__numberofcomps  int64 3
    profile__c_state     (time, NPZD) float64 0.0 1.0 2.0 ... 2.47e+41 4.94e+41